In [1]:
import numpy as np
from ns3gym import ns3env
from DQN_model import DeepQNetwork
from DQN_model import Eval_Model
from DQN_model import Target_Model
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
learning_rate = 0.01
reward_decay = 0.9
e_greedy = 0.9
replace_target_iter = 100
memory_size = 500
batch_size = 32
training_episodes = 20
testing_episodes = 10
max_chosen = 3
training_repeat_chosen = []
testing_repeat_chosen = []
episode_time = 300

In [3]:
eval_model = Eval_Model(num_actions=32)
target_model = Target_Model(num_actions=32)
RL = DeepQNetwork(32, 3, 32,
                  eval_model, target_model, learning_rate, reward_decay, e_greedy, 
                  replace_target_iter, memory_size, batch_size)

In [4]:
for episode in range(1):
    print ("-----------------------episodes: ", episode, " -----------------------")
    stepIdx = 0
    repeat_chosen_counter = 0
    
    _obs = np.zeros(32,dtype=int)
    
    
    
    while True:
        #print ("\nstepIdx:",stepIdx)
        stepIdx += 1
        reward_all = []
        
        action = np.array(RL.choose_action(_obs, False))

        
        # Using the queuing bytes to decide how many data slot would be choose
        #action_num = 0 if queueBytes == 0 else min(max(0, int(queueBytes/6250 - 0.3)) + 1, 3)
        #action_num = 2
        #action[action_num:] = [-1] * (max_chosen-action_num)
        action = action[:-1]
        
        #print("action: ",action)
        
        obs = _obs
        for a in action:
            if obs[a] == 0:
                obs[a] = -1
                reward_all.append(100)
        
        #print ("obs: ",obs)
        #print ("reward:", reward_all)
        
        
        for act, r in zip(action,reward_all):
                
            RL.store_transition(_obs, act, r, obs)
            
            if r <= -100:
                repeat_chosen_counter += 1

                
        if stepIdx % 16 == 0:
            obs = np.zeros(32,dtype=int)
        
        #if stepIdx % 100 == 0:
        #    print ("stepIdx: ",stepIdx)
        
        if (stepIdx > 5000) and (stepIdx % 5 == 0):
            RL.learn()
            
        # swap observation
        _obs = obs
        
        
        if stepIdx == 120000:
            print("Step: ", stepIdx)
            print ("done")
            print ("Repeat_chosen_counter :", repeat_chosen_counter)
            RL.learn()
            
            training_repeat_chosen.append(repeat_chosen_counter)
            break
            
        #break
        


-----------------------episodes:  0  -----------------------


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-23f108e2b230>", line 51, in <module>
    RL.learn()
  File "/tf/ns3-gym/scratch/tdma-rl/DQN_model.py", line 110, in learn
    q_next = self.target_model.predict(batch_memory[:, -self.params['n_states']:])
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 87, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1181, in predict
    model=self)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/data_adapter.py", line 1124, in __init__
    model=model)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/data_adapter.py", line 363, in __init__
    indices_dataset = indices_d

KeyboardInterrupt: 